In [ ]:
import os
from flask import Flask, flash, request, redirect, url_for,Response
from werkzeug.utils import secure_filename
import keras
import numpy as np
import pandas as pd
import PIL
from keras.models import load_model
from PIL import Image
import cv2 as cv
from keras.applications import vgg16
from keras.preprocessing.image import load_img,img_to_array
from keras.models import Model
from keras.applications.imagenet_utils import preprocess_input
import pickle
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import jsonpickle
import json
path_to_model = 'weights.30.h5'
model = load_model(path_to_model)
modelConfiguration = "/Users/infinitejock/learnopencv/ObjectDetection-YOLO/yolov3.cfg";
modelWeights = "/Users/infinitejock/learnopencv/ObjectDetection-YOLO/yolov3.weights";
vgg_model = vgg16.VGG16(weights='imagenet')
feat_extractor = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)
net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)
with open ('/Users/infinitejock/Product-recommendation-based-on-visual-similarity/outfile', 'rb') as fp:
    itemlist = pickle.load(fp)
feat = np.load('/Users/infinitejock/Product-recommendation-based-on-visual-similarity/feat.npy')
classesFile = '/Users/infinitejock/learnopencv/ObjectDetection-YOLO/coco.names'
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')
    
    
UPLOAD_FOLDER = '/Users/infinitejock/Desktop/'
ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])

app = Flask(__name__,static_url_path='/static')
app.secret_key = os.urandom(24)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods = ['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        dic={}
        file = request.files['file']
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))

        save_path=os.path.join(app.config['UPLOAD_FOLDER'], filename)


        pred_mod=scene_classifier(model,net,feat_extractor,itemlist,feat,classes)
        classs=pred_mod.predict_class(save_path)
        classs=classs[0][0]
        subclass=pred_mod.predict_objects(save_path)
        dic['class']=classs
        dic['objects']=subclass
        dic['img']=save_path
        response_pickled = jsonpickle.encode(dic)
        print(response_pickled)
        response=app.response_class(response=json.dumps(dic), status=200, mimetype="application/json")
        #return response
        page='''
    <!doctype html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css">
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>
  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/js/bootstrap.min.js"></script>
  </head>
  
    <title>Result</title>
    <h1>Room Identified as :'''+dic['class']+'''</h1>
    <img src="/static/test.jpg" width="850" height="450"/>
    <h2>Objects</h2>'''
        for o in dic['objects']:
            page+='''<button type="button" class="btn btn-info" data-toggle="collapse" data-target="#'''+o['class']+'''">'''+o['class']+'''</button>
  <div id="'''+o['class']+'''" class="collapse">'''
            page+='''<h2>'''+o['class']+'''</h2>'''
            for i in o['similar']:
                page+='''<img src="/static/'''+"/".join(i.split('/')[-2:])+'''"width="250" height="250"/>'''
            page+='''</div>'''
        #page+='''</div>'''
        return  page
    return '''
    <!doctype html>
    <title>Upload new File</title>
    <body background="http://thetheme.io/thesaasx/wp-content/themes/thesaasx/assets/img/vector/21.png">
    
    <div align="center" style="border:1px solid red">
    <img src="static/logo.png" width="100" height="50">
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <p><input type=file name=file>
         <input type=submit value=Upload>
    
    </form>
    </div>
    '''


class scene_classifier:
    
    def __init__(self,model,net,feat_ext,itemlist,feat,classes):
        self.label_lst = [
        "Living room", "Kitchen", "Hallway", "Bathroom", "Bedroom", "Balcony",
        "House", "Facade", "Sauna", "Terrace", "Separate WC", "Floor plan",
        "Dining space", "Yard", "Other", "Upstairs", "Open kitchen",
        "Downstairs", "Room", "Clothes room", "Utility room", "WC", "Alcove",
        "Fireplace"
    ]
        #self.path_to_model = 'weights.30.h5'
        self.model = model
        self.confThreshold = 0.5
        self.nmsThreshold = 0.4
        self.inpWidth = 416
        self.inpHeight = 416
        self.classes = None
        self.itemlist=itemlist
        self.feat=feat
        self.classes=classes
        self.feat_extractor = feat_ext
        self.net=net
    def getOutputsNames(self,net):
        layersNames = net.getLayerNames()
        return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    
    def drawPred(self,frame,classId, conf, left, top, right, bottom,box):
        label = self.classes[classId]
        if label !='book':
            #cv.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
            img=frame[top:bottom,left:right]
            files=self.predict_similar(img)
            cv.imwrite('/Users/infinitejock/Desktop/'+label+'.jpg',img)
            return files
    # Remove the bounding boxes with low confidence using non-maxima suppression
    def postprocess(self,frame, outs):
        frameHeight = frame.shape[0]
        frameWidth = frame.shape[1]

        # Scan through all the bounding boxes output from the network and keep only the
        # ones with high confidence scores. Assign the box's class label as the class with the highest score.
        classIds = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                classId = np.argmax(scores)
                confidence = scores[classId]
                if confidence > self.confThreshold:
                    center_x = int(detection[0] * frameWidth)
                    center_y = int(detection[1] * frameHeight)
                    width = int(detection[2] * frameWidth)
                    height = int(detection[3] * frameHeight)
                    left = int(center_x - width / 2)
                    top = int(center_y - height / 2)
                    classIds.append(classId)
                    confidences.append(float(confidence))
                    boxes.append([left, top, width, height])


        indices = cv.dnn.NMSBoxes(boxes, confidences, self.confThreshold, self.nmsThreshold)
        subclass=[]
        
        for i in indices:
            temp={}
            i = i[0]
            box = boxes[i]
            left = box[0]
            top = box[1]
            width = box[2]
            height = box[3]
            
            files=self.drawPred(frame,classIds[i], confidences[i], left, top, left + width, top + height,box)
            if self.classes[classIds[i]] !='book':
                cv.rectangle(frame, (left, top), (left + width, top + height), (255, 178, 50), 3)
                temp['class']=self.classes[classIds[i]]
                temp['similar']=files
                temp['rect']=[left,top,width,height]
                subclass.append(temp)
                label=self.classes[classIds[i]]
                labelSize, baseLine = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                top = max(top, labelSize[1])
                cv.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (255, 255, 255), cv.FILLED)
                cv.putText(frame, label, (left, top), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1)
        cv.imwrite('/Users/infinitejock/apartment_room_photos_classification/cnn/static/test.jpg',frame)
        return subclass
        

    def predict_class(self,path):
        img = Image.open(path).resize((299, 299))
        img_array = np.array(img) / 255
        pred = self.model.predict(np.array([img_array]))
        di = dict(zip(self.label_lst, pred[0]))
        val=sorted(di.items(), key=lambda kv: kv[1], reverse=True)[:1]
        return val
    def predict_objects(self,path):
        img=cv.imread(path)
        blob = cv.dnn.blobFromImage(img, 1/255, (self.inpWidth, self.inpHeight), [0,0,0], 1, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.getOutputsNames(self.net))
        subclass=self.postprocess(img, outs)
        return subclass
    def predict_similar(self,obj):
        try:
            imgs_model_width, imgs_model_height = 224, 224
            obj=cv.resize(obj, (224, 224)) 
            nb_closest_images = 5

            numpy_image = img_to_array(obj)
            image_batch = np.expand_dims(numpy_image, axis=0)
            processed_image = preprocess_input(image_batch.copy())
            img_features = self.feat_extractor.predict(processed_image)
            ab=[]
            for i in range(len(self.feat)):
                dist=cosine_similarity([self.feat[i]],img_features)
                ab.append(dist[0][0])
            arr = np.array(ab)
            idx=arr.argsort()[-5:]
            #idx=sorted(range(len(ab)), key=lambda i: ab[i])[-5:]
            files=[]
            for i in idx:
                files.append(self.itemlist[i])
            return files
        except:
            return []
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8300,debug=False)

 * Running on http://0.0.0.0:8300/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Mar/2019 12:08:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:33] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:33] "GET /static/clock/img248.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:33] "GET /static/objects/701.524.67.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:34] "GET /static/clock/img7.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:34] "GET /static/clock/img172.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:34] "GET /static/clock/img30.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:34] "GET /static/chair/390.108.90.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:34] "GET /static/objects/402.919.26.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:34] "GET /static/objects/002.919.28.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:34] "GET /static/objects/902.797.19.jpg HTTP/1.1" 200 -


{"class": "Bedroom", "img": "/Users/infinitejock/Desktop/ikea-a-bed-that-folds-away-to-be-a-sofa-by-day__1364309472525-s4.jpg", "objects": [{"class": "bed", "rect": [99, 416, 459, 305], "similar": ["/Users/infinitejock/apartment_room_photos_classification/cnn/static/room_scenes/ikea-unwind-in-warm-brown-and-cool-blue__1364308421559-s4.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/bed/images_157.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/bed/202.312.26.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/room_scenes/ikea-a-bed-that-folds-away-to-be-a-sofa-by-day__1364309472525-s4_yolo_out_py.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/bed/images_054.jpg"]}, {"class": "clock", "rect": [648, 127, 88, 81], "similar": ["/Users/infinitejock/apartment_room_photos_classification/cnn/static/clock/img172.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/clock

127.0.0.1 - - [17/Mar/2019 12:09:34] "GET /static/objects/403.057.87.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:34] "GET /static/objects/303.097.81.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:34] "GET /static/plant_pot/images_119.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:34] "GET /static/plant_pot/images_047.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:09:34] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Mar/2019 12:13:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 12:14:19] "POST / HTTP/1.1" 200 -


{"class": "Living room", "img": "/Users/infinitejock/Desktop/ikea-a-fresh-way-to-keep-your-clothes-tidy__1364315963128-s4.jpg", "objects": [{"class": "bed", "rect": [-8, 306, 347, 415], "similar": []}]}


127.0.0.1 - - [17/Mar/2019 12:14:29] "GET / HTTP/1.1" 200 -
10.2.41.99 - - [17/Mar/2019 12:14:58] "POST / HTTP/1.1" 200 -
10.2.41.99 - - [17/Mar/2019 12:14:58] "GET /static/objects/700.914.12.jpg HTTP/1.1" 200 -


...

127.0.0.1 - - [17/Mar/2019 17:58:56] "GET /static/chair/301.165.51.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 17:58:56] "GET /static/chair/images_092.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 17:58:56] "GET /static/chair/images_006.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 17:58:56] "GET /static/chair/images_106.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 17:58:56] "GET /static/chair/images_022.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 17:58:56] "GET /static/dining%20table/690.403.53.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 17:58:56] "GET /static/chair/images_112.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 17:58:56] "GET /static/chair/images_055.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 17:58:56] "GET /static/plant_pot/images_130.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 17:58:56] "GET /static/plant_pot/images_027.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 17:58:56] "GET /static/plant_pot/images_134.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/

{"class": "Bedroom", "img": "/Users/infinitejock/Desktop/ikea-a-bed-that-folds-away-to-be-a-sofa-by-day__1364309472525-s4.jpg", "objects": [{"class": "bed", "rect": [99, 416, 459, 305], "similar": ["/Users/infinitejock/apartment_room_photos_classification/cnn/static/room_scenes/ikea-unwind-in-warm-brown-and-cool-blue__1364308421559-s4.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/bed/images_157.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/bed/202.312.26.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/room_scenes/ikea-a-bed-that-folds-away-to-be-a-sofa-by-day__1364309472525-s4_yolo_out_py.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/bed/images_054.jpg"]}, {"class": "clock", "rect": [648, 127, 88, 81], "similar": ["/Users/infinitejock/apartment_room_photos_classification/cnn/static/clock/img172.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/clock

127.0.0.1 - - [17/Mar/2019 22:31:39] "GET /static/bed/images_157.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 22:31:39] "GET /static/room_scenes/ikea-unwind-in-warm-brown-and-cool-blue__1364308421559-s4.jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2019 22:31:39] "GET /static/test.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:14:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:14:50] "GET /static/logo.png HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:14:50] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/chair/903.347.30.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/chair/902.396.67.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/objects/502.603.97.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/chair/images_073.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/objects/802.835.85.jpg HTTP/1.1" 

{"class": "Kitchen", "img": "/Users/infinitejock/Desktop/ikea-a-contemporary-kitchen-for-big-chefs-and-little-helpers__1364316013714-s4.jpg", "objects": [{"class": "chair", "rect": [737, 432, 221, 209], "similar": ["/Users/infinitejock/apartment_room_photos_classification/cnn/static/objects/502.603.97.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/chair/902.396.67.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/chair/images_092.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/chair/903.347.30.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/chair/images_073.jpg"]}, {"class": "bowl", "rect": [899, 355, 106, 45], "similar": ["/Users/infinitejock/apartment_room_photos_classification/cnn/static/objects/802.835.85.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/objects/201.375.73.jpg", "/Users/infinitejock/apartment_room_photos_classification/cnn/static/objec

127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/chair/402.177.95.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/chair/101.150.67.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/chair/901.011.13.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/dining%20table/images_121.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/test.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/objects/202.684.65.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/chair/images_092.jpg HTTP/1.1" 304 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/objects/101.189.71.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/chair/401.536.99.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/dining%20table/images_152.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:29] "GET /static/chair/images_105.jpg HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2019 12:15:

In [3]:
i='/Users/infinitejock/apartment_room_photos_classification/cnn/static/plant_pot/images_119.jpg'

In [15]:
"/".join(i.split('/')[-2:])

'plant_pot/images_119.jpg'